# Imports

In [1]:
import numpy as np
import unittest

# Define Components

## Vector Class

In [40]:
class Vector2d:
    def __init__(self, coords=None):
        ''' coords is array of size two
        if argument coords is not specified, then Vector2d will have random coordinates
        '''
        if coords is None:
            self._coords = np.random.random(2)
        else:
            self._coords = np.array(coords)
            
    def get_x(self):
        return self._coords[0]
    
    def set_x(self, value):
        self._coords[0] = value

    def get_y(self):
        return self._coords[1]
    
    def set_y(self, value):
        self._coords[1] = value
    
    x = property()
    y = property()

    x = x.getter(get_x)
    x = x.setter(set_x)

    y = y.getter(get_y)
    y = y.setter(set_y)

    def __str__(self):
        return '[{0:,.3f}, {1:,.3f}] Vector2d'.format(self.x, self.y)
    
    def __repr__(self):
        return '[{0}, {1}] Vector2d'.format(self.x, self.y)
    
    def __eq__(self, other):
        return all(self._coords == other._coords)
        
    def __add__(self, other):
        return Vector2d(self._coords + other._coords)
    

## Environment Class

## Atom Class

## Moleculalr Dynamics Class

# Launching

# Unit Testing

## Vector Class

In [47]:
class VectorTest(unittest.TestCase):
    def setUp(self):
        self.arr = [Vector2d(), Vector2d(), Vector2d()]
        self.v2 = Vector2d([3,4.4])
        
    def test_coords(self):
        self.assertEqual(np.testing.assert_array_equal(np.array([3,4.4]), self.v2._coords), None)
        
    def test_random_vectors(self):
        print('Make sure that the vectors are different:\n',[v._coords for v in self.arr])
        self.assertEqual(9,9)


class VectorGetSet(unittest.TestCase):
    def setUp(self):
        self.v = Vector2d([10.2, 4.154])
    
    def test_set(self):
        self.v.x = 3
        self.v.y = 5
        self.assertEqual(np.testing.assert_array_equal(np.array([3,5]), self.v._coords), None)

    def test_get(self):
        self.assertEqual(self.v.x, 10.2)
        self.assertEqual(self.v.y, 4.154)



class VectorTestStrRepr(unittest.TestCase):
    def setUp(self):
        self.v = Vector2d([22.351355, -44.343143])
        
    def test_str(self):
        self.assertEqual(str(self.v), '[22.351, -44.343] Vector2d')
        
    def test_repr(self):
        self.assertEqual((self.v.__repr__()), '[22.351355, -44.343143] Vector2d')


class VectorEquality(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([22.351355, -44.343143])
        self.v2 = Vector2d([22.351355, -44.343143])
        self.v3 = Vector2d([33.3, -22.343143])
        
    def test_eq(self):
        self.assertEqual(self.v1, self.v2)
        
    def test_neq(self):
        self.assertNotEqual(self.v1, self.v3)

class VectorAddition(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([22.3, -44])
        self.v2 = Vector2d([2,4])
        
    def test_add(self):
        result = Vector2d([24.3, -40])
        self.assertEqual(self.v1 + self.v2, result)

        
unittest.main(argv=['first-arg-is-ignored'], exit=False);

.........

Make sure that the vectors are different:
 [array([0.44332233, 0.30750914]), array([0.30921199, 0.03306565]), array([0.83471502, 0.88774446])]



----------------------------------------------------------------------
Ran 9 tests in 0.010s

OK


## Environment Class

## Atom Class

## Moleculalr Dynamics Class